In [1]:
import numpy as np
import os
import networkx as nx
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from numpy import dot
from numpy.linalg import norm


from collections import Counter
import matplotlib.pyplot as plt



unzip:  cannot find or open cora (extract.me).zip, cora (extract.me).zip.zip or cora (extract.me).zip.ZIP.


In [2]:
all_data = []
all_edges = []

for root,dirs,files in os.walk('./cora'):
    for file in files:
        if '.content' in file:
            with open(os.path.join(root,file),'r') as f:
                all_data.extend(f.read().splitlines())
        elif 'cites' in file:
            with open(os.path.join(root,file),'r') as f:
                all_edges.extend(f.read().splitlines())

                
#random_state = 42
#all_data = shuffle(all_data,random_state=random_state)

In [4]:
categories =  ['Reinforcement_Learning', 'Theory', 'Case_Based', 'Genetic_Algorithms', 'Probabilistic_Methods', 'Neural_Networks', 'Rule_Learning']
sorted(categories)
label_encoder = {}
i = 0
for cat in sorted(categories):
  label_encoder[cat] = i
  i +=1
label_encoder


{'Case_Based': 0,
 'Genetic_Algorithms': 1,
 'Neural_Networks': 2,
 'Probabilistic_Methods': 3,
 'Reinforcement_Learning': 4,
 'Rule_Learning': 5,
 'Theory': 6}

In [5]:
#parse the data
labels = []
nodes = []
X = []
element_to_ind  = {}

for i,data in enumerate(all_data):
    elements = data.split('\t')
    labels.append(label_encoder[elements[-1]])
    X.append(elements[1:-1])
    nodes.append(elements[0])
    element_to_ind[elements[0]]= i
X = np.array(X,dtype=int)
N = X.shape[0] #the number of nodes
F = X.shape[1] #the size of node features
print('X shape: ', X.shape)


#parse the edge
edge_list=[]
for edge in all_edges:
    e = edge.split('\t')
    edge_list.append((e[0],e[1]))

print('\nNumber of nodes (N): ', N)
print('\nNumber of features (F) of each node: ', F)
print('\nCategories: ', set(labels))

num_classes = len(set(labels))
print('\nNumber of classes: ', num_classes)


X shape:  (2708, 1433)

Number of nodes (N):  2708

Number of features (F) of each node:  1433

Categories:  {0, 1, 2, 3, 4, 5, 6}

Number of classes:  7


In [7]:
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edge_list)
G = nx.relabel_nodes(G, element_to_ind)
print('Graph info: ', nx.info(G))

Graph info:  Graph with 2708 nodes and 5278 edges


/tmp/ipykernel_645724/1788189256.py:5: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print('Graph info: ', nx.info(G))


In [8]:
nodes = list(G.nodes)
print(len(nodes))
list(G.neighbors(0))

2708


[258, 544, 8, 435, 14]

In [9]:
df = pd.DataFrame(list(zip(nodes, labels,X)),columns =['node', 'label','features'])
print(len(df))
df.head()

2708


,node,label,features
0,0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,2,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [10]:
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G = G.subgraph(Gcc[0])
gcc_nodes = list(G.nodes)

In [11]:
df = df.loc[df['node'].isin(gcc_nodes)]
df['node'] = list(range(len(df))) #rename nodes 
df.head()

,node,label,features
0,0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,2,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [12]:
train = df.groupby('label', group_keys=False).apply(lambda x: x.sample(20))
G = nx.relabel_nodes(G, df['node'])

In [13]:
def create_transition_matrix(g):
    vs = list(g.nodes)
    n = len(vs)
    adj = nx.adjacency_matrix(g)
    transition_matrix = adj/adj.sum(axis=1)

    return transition_matrix

In [16]:
def random_walk(g, num_steps, start_node, transition_matrix = None):
  if transition_matrix is None:
    transition_matrix = create_transition_matrix(g)
  #perform a random walk
  v = start_node
  walked_nodes = range(transition_matrix.shape[0])

  for i in range(num_steps):
      PMF = transition_matrix[v,]
      v = np.random.choice(walked_nodes, p = np.ravel(PMF, order = 'C'))

  return v


In [17]:
seeds_dict = {predicted:list(train[train['label'] == predicted]['node']) for predicted in range(7)}

def random_walk_with_teleportation(g, num_steps, start_node,tp,predicted, transition_matrix = None):
  #modify random walk code to add teleportation.
  #you can only teleport to a node belonging to the same class as the start node
  if transition_matrix is None:
    transition_matrix = create_transition_matrix(g)

  v = start_node
  walked_nodes = range(transition_matrix.shape[0])
  seeds = seeds_dict[predicted]

  for i in range(num_steps):
      run = np.random.uniform(low=0.0, high=1.0, size=None)
      if run < tp:
          v = np.random.choice(seeds)
      else:
          PMF = transition_matrix[v,]
          v = np.random.choice(walked_nodes, p = np.ravel(PMF, order='C'))

  return v

  #return v


In [19]:
#pagerank. NO teleportation, NO tfidf. 
transition_matrix = create_transition_matrix(G)

num_samples = 1000  
num_walk_steps = 100

visiting_freq_label = []
for i in range(transition_matrix.shape[0]):
  visiting_freq_label.append([0,0,0,0,0,0,0])

visiting_freq = [0 for i in range(transition_matrix.shape[0])]


for train_node,predicted in zip(train['node'],train['label']):
  #print (train_node,predicted)
  for i in range(num_samples):
      start_point = train_node
      end_node = random_walk(G, num_walk_steps, start_point, transition_matrix)
      visiting_freq_label[end_node][predicted] += 1
      visiting_freq[end_node] +=1

/tmp/ipykernel_645724/2140906970.py:4: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(g)


In [20]:
count = 0 #these many nodes remain unvisited. 
for vf in visiting_freq:
  if vf ==0:
    count+=1
print('Pagerank without teleportation')
print('unvisited = ', count)
visiting_freq_label = np.asarray(visiting_freq_label)
preds = np.argmax(visiting_freq_label,axis = 1)
print(classification_report(df['label'], preds))
accuracy_score(df['label'], preds)

Pagerank without teleportation
unvisited =  0
              precision    recall  f1-score   support

           0       0.31      0.52      0.39       285
           1       0.39      0.45      0.42       406
           2       0.49      0.25      0.33       726
           3       0.68      0.57      0.62       379
           4       0.10      0.14      0.12       214
           5       0.11      0.24      0.15       131
           6       0.26      0.19      0.22       344

    accuracy                           0.34      2485
   macro avg       0.33      0.34      0.32      2485
weighted avg       0.40      0.34      0.35      2485



0.344466800804829

In [23]:
#pagerank. WITH telportation, without tfidf 

#repeat above expeiment but this time use the teleportation random walk 

#get metrics
def page_rank_teleportation(graph, tele_prob):
  G = graph 
  transition_matrix = create_transition_matrix(G)

  num_samples = 1000  
  num_walk_steps = 100

  visiting_freq_label = []
  for i in range(transition_matrix.shape[0]):
    visiting_freq_label.append([0,0,0,0,0,0,0])

  visiting_freq = [0 for i in range(transition_matrix.shape[0])]

  for train_node,predicted in zip(train['node'],train['label']):
        for i in range(num_samples):
            start_point = train_node
            end_node = random_walk_with_teleportation(G, num_walk_steps, start_point, tele_prob, predicted, transition_matrix)
            end_node = int(end_node)
            visiting_freq_label[end_node][predicted] += 1
            visiting_freq[end_node] +=1

  count = 0 #these many nodes remain unvisited. 
  for vf in visiting_freq:
      if vf ==0:
          count+=1

  print("Pagerank Random Walk with teleportation, No TF-IDF ")
  print('unvisited = ', count)
  visiting_freq_label = np.asarray(visiting_freq_label)
  preds = np.argmax(visiting_freq_label,axis = 1)
  print(classification_report(df['label'], preds))
  print("Accuracy: ", accuracy_score(df['label'], preds))

In [24]:
tele_probs = [0.1, 0.2, 0.3 ]
for tele_prob in tele_probs:
  print("teleportation probability: ", tele_prob)
  page_rank_teleportation(G, tele_prob)
  print("------------")

teleportation probability:  0.1


/tmp/ipykernel_645724/2140906970.py:4: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(g)


Pagerank Random Walk with teleportation, No TF-IDF 
unvisited =  34
              precision    recall  f1-score   support

           0       0.65      0.76      0.70       285
           1       0.81      0.91      0.86       406
           2       0.82      0.52      0.64       726
           3       0.78      0.79      0.79       379
           4       0.60      0.82      0.70       214
           5       0.50      0.81      0.62       131
           6       0.61      0.61      0.61       344

    accuracy                           0.71      2485
   macro avg       0.68      0.75      0.70      2485
weighted avg       0.73      0.71      0.70      2485

Accuracy:  0.7070422535211267
------------
teleportation probability:  0.2


/tmp/ipykernel_645724/2140906970.py:4: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(g)


Pagerank Random Walk with teleportation, No TF-IDF 
unvisited =  74
              precision    recall  f1-score   support

           0       0.57      0.76      0.65       285
           1       0.81      0.88      0.84       406
           2       0.82      0.52      0.63       726
           3       0.76      0.75      0.76       379
           4       0.60      0.82      0.69       214
           5       0.55      0.76      0.64       131
           6       0.60      0.63      0.61       344

    accuracy                           0.69      2485
   macro avg       0.67      0.73      0.69      2485
weighted avg       0.72      0.69      0.69      2485

Accuracy:  0.6933601609657948
------------
teleportation probability:  0.3


/tmp/ipykernel_645724/2140906970.py:4: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(g)


Pagerank Random Walk with teleportation, No TF-IDF 
unvisited =  157
              precision    recall  f1-score   support

           0       0.46      0.73      0.57       285
           1       0.81      0.87      0.84       406
           2       0.81      0.46      0.59       726
           3       0.77      0.73      0.75       379
           4       0.57      0.81      0.67       214
           5       0.54      0.76      0.63       131
           6       0.60      0.59      0.60       344

    accuracy                           0.66      2485
   macro avg       0.65      0.71      0.66      2485
weighted avg       0.70      0.66      0.66      2485

Accuracy:  0.6631790744466801
------------


In [26]:
vs = list(G.nodes)
n = len(vs)
adj = nx.adjacency_matrix(G)
transition = np.zeros((len(G.nodes), len(G.nodes)))

for n1 in nodes:
  for n2 in nodes:
    if G.has_edge(n1, n2):
      f1 = list(df['features'])[n1]
      f2 = list(df['features'])[n2]
      cos_sim = np.dot(f1, f2)/(norm(f1)*norm(f2))
      transition[n1,n2] = np.exp(cos_sim)
    # if there is an edge between n1 and n2:
      # cos_sim = compute cosine similarity between features of n1 and n2
      # transition[n1,n2] = np.exp(cos_sim) #neumerator of softmax. #why do we need softmax?  
# divide the values in transition by denominator of softmax. how will you do this? 
transition = transition/transition.sum(axis=1, keepdims=True)

/tmp/ipykernel_645724/2329422368.py:3: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(G)


In [27]:
def page_rank_teleportation_tfiDF(graph, tele_prob, transition_matrix):
  G = graph 
#   transition_matrix = create_transition_matrix(G)

  num_samples = 1000  
  num_walk_steps = 100

  visiting_freq_label = []
  for i in range(transition_matrix.shape[0]):
    visiting_freq_label.append([0,0,0,0,0,0,0])

  visiting_freq = [0 for i in range(transition_matrix.shape[0])]

  for train_node,predicted in zip(train['node'],train['label']):
        for i in range(num_samples):
            start_point = train_node
            if tele_prob == 0:
                end_node = random_walk(G, num_walk_steps, start_point, transition_matrix)
            else:
                end_node = random_walk_with_teleportation(G, num_walk_steps, start_point, tele_prob, predicted, transition_matrix)
            end_node = int(end_node)
            visiting_freq_label[end_node][predicted] += 1
            visiting_freq[end_node] +=1

  count = 0 #these many nodes remain unvisited. 
  for vf in visiting_freq:
      if vf ==0:
          count+=1

  print("Pagerank Random Walk with teleportation, with TF-IDF ")
  print('unvisited = ', count)
  visiting_freq_label = np.asarray(visiting_freq_label)
  preds = np.argmax(visiting_freq_label,axis = 1)
  print(classification_report(df['label'], preds))
  print('accuracy: ', accuracy_score(df['label'], preds) )

In [28]:
tele_probs = [0, 0.2, 0.3 ]
for tele_prob in tele_probs:
  print("teleportation probability: ", tele_prob)
  page_rank_teleportation_tfiDF(G, tele_prob, transition)
  print("------------")

teleportation probability:  0
Pagerank Random Walk with teleportation, with TF-IDF 
unvisited =  1
              precision    recall  f1-score   support

           0       0.36      0.58      0.45       285
           1       0.46      0.51      0.48       406
           2       0.47      0.25      0.33       726
           3       0.64      0.55      0.59       379
           4       0.14      0.19      0.16       214
           5       0.10      0.23      0.14       131
           6       0.24      0.20      0.22       344

    accuracy                           0.36      2485
   macro avg       0.34      0.36      0.34      2485
weighted avg       0.40      0.36      0.37      2485

accuracy:  0.3617706237424547
------------
teleportation probability:  0.2
Pagerank Random Walk with teleportation, with TF-IDF 
unvisited =  86
              precision    recall  f1-score   support

           0       0.57      0.75      0.65       285
           1       0.81      0.92      0.86       

In [29]:
print("teleportation probability: ", 0.1)
page_rank_teleportation_tfiDF(G, 0.1, transition)
print("------------")

teleportation probability:  0.1
Pagerank Random Walk with teleportation, with TF-IDF 
unvisited =  29
              precision    recall  f1-score   support

           0       0.65      0.77      0.71       285
           1       0.82      0.92      0.87       406
           2       0.82      0.52      0.64       726
           3       0.80      0.78      0.79       379
           4       0.61      0.83      0.70       214
           5       0.49      0.82      0.62       131
           6       0.60      0.61      0.61       344

    accuracy                           0.71      2485
   macro avg       0.69      0.75      0.70      2485
weighted avg       0.73      0.71      0.71      2485

accuracy:  0.7098591549295775
------------
